[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/MGP/blob/main/GP/q1_julia.ipynb)

# Um Problema de Mix de Produção

$$\max z = 0,3x_1 + 0,25x_2 + 0,2x_3$$
                                            sujeito a:
$$0,22x_1 + 0,52x_2 + 0,74x_3 \leq 9600000$$
$$0,52x_1 + 0,34x_2 + 0,20x_3 \leq 4800000$$
$$0,28x_1 + 0,14x_2 + 0,06x_3 \leq 2200000$$ 
$$x_3 \geq 16x_1$$
$$x_2 \leq 600000$$
$$x_i \geq 0,   \forall i \in \{1,2,3\}$$
  

## Usando Julia no Google Colab

**Para usar julia no colab, precisamos instalar a linguagem previamente.**

Se esse for o caso, run a célula abaixo, template produzido por: https://colab.research.google.com/github/ageron/julia_notebooks/blob/master/Julia_Colab_Notebook_Template.ipynb#scrollTo=GIeFXS0F0zww

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

In [ ]:
using Pkg
Pkg.add(["JuMP", "GLPK", "DataFrames"])

In [ ]:
using JuMP, GLPK

In [ ]:
M = Model(GLPK.Optimizer);

@variable(M, x[1:3] >=0)

@constraints(M, begin
                0.22 * x[1]+ 0.52 * x[2] + 0.74 * x[3] <= 9600000 
                0.5*x[1]+ 0.34 * x[2] + 0.2 * x[3] <= 4800000
                0.28*x[1]+ 0.14 * x[2] + 0.06*x[3] <= 2200000
                x[3] >= 16*x[1]
                x[2] <= 600000
                end
)

@objective(M, Max, 0.3*x[1] + 0.25*x[2] + 0.2*x[3])

println(M) # Imprimir o modelo na tela

optimize!(M);
z = objective_value(M); x_ = value.(x);
println("Valor alcançado na função objetivo: $z\nx=$x_")

### Inserção dos Dados


In [ ]:
dados = DataFrame(tipo_prod= ["verde","azul","comum"], pura=[0.22, 0.52, 0.74], octana = [0.5, 0.34, 0.2], aditivo = [0.28, 0.14, 0.06]);

In [ ]:
dados = stack(dados, [:pura, :octana, :aditivo]);
rename!(dados, [:variable, :value] .=> [:mp, :comp]);
dados

In [ ]:
comp_dict = Dict();
for i =1:nrow(dados)
    push!(comp_dict, (dados.tipo_prod[i], dados.mp[i]) => dados.comp[i])
end
comp_dict

In [ ]:
tipo_prod= ["verde","azul","comum"];                  
mp = ["pura", "octana", "aditivo"];
capacidades = Dict("pura" => 9600000, "octana" => 4800000, "aditivo" => 2200000);
lucros = Dict("verde" => 0.3, "azul" => 0.25, "comum" => 0.2);
capacidade_azul = 600000;

### Criando o Modelo

In [ ]:
M = Model(GLPK.Optimizer)

@variable(M, x[tipo_prod] >=0)

@constraint(M, R_capacidades[j in mp], sum(x[i] * comp_dict[i, j] for i in tipo_prod) <= capacidades[j])
@constraints(M, begin
                x["comum"] >= 16 * x["verde"]
                x["azul"] <= capacidade_azul
end)

@objective(M, Max, sum(x[i] * lucros[i] for i in tipo_prod))

println(M) # Imprimir o modelo na tela

optimize!(M);

In [ ]:
z = objective_value(M); 
x_ = value.(x);

println("Valor alcançado na função objetivo: $z\nx=$x_")